In [55]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import os
import xgboost as xgb

In [46]:
# parent directory
parent_dir = Path.cwd().parent

# path to the dataset
data_path = os.path.join(parent_dir ,'data','raw','weekly-train-1.csv')

# reading a CSV file into a DataFrame
df = pd.read_csv(data_path)

In [47]:
df.head()

,PRODUCT_DESC,PRODUCT_ID,DATE,TOTAL_SALES,PRICE,IS_COUPON,IS_DISPLAY,IS_FEATURE,IS_PROMO,IS_INDEPENDENCE_DAY,IS_PRE_CHRISTMAS,IS_SUPER_BOWL_SUNDAY,IS_PRE_THANKSGIVING,IS_EASTER_SUNDAY,IS_LABOR_DAY,IS_MEMORIAL_DAY,IS_THANKSGIVING,IS_CHRISTMAS_DAY
0,Yogurt_Greek,3450014449,2020-12-28,20461.337740,5.667959,False,True,False,True,False,False,False,False,False,False,False,False,False
1,Yogurt_Greek,3450014449,2021-01-04,20366.054602,5.666967,False,True,False,True,False,False,False,False,False,False,False,False,False
2,Yogurt_Greek,3450014449,2021-01-11,18072.334581,5.667983,False,False,False,False,False,False,False,False,False,False,False,False,False
3,Yogurt_Greek,3450014449,2021-01-18,18119.333530,5.667809,False,False,False,False,False,False,False,False,False,False,False,False,False
4,Yogurt_Greek,3450014449,2021-01-25,18867.597360,5.040478,False,False,False,True,False,False,False,False,False,False,False,False,False


In [48]:
# checking the data type of the 'DATE' column
df['DATE'].dtype.name

'str'

In [49]:
# converting the 'DATE' column to datetime format
df['DATE'] = pd.to_datetime(df['DATE'])
print(df['DATE'].dtype.name)

datetime64[us]


In [50]:
# check if any rows have missing values
null_rows = df[df.isnull().any(axis=1)]
null_rows

,PRODUCT_DESC,PRODUCT_ID,DATE,TOTAL_SALES,PRICE,IS_COUPON,IS_DISPLAY,IS_FEATURE,IS_PROMO,IS_INDEPENDENCE_DAY,IS_PRE_CHRISTMAS,IS_SUPER_BOWL_SUNDAY,IS_PRE_THANKSGIVING,IS_EASTER_SUNDAY,IS_LABOR_DAY,IS_MEMORIAL_DAY,IS_THANKSGIVING,IS_CHRISTMAS_DAY
